In [79]:
import requests
import pandas as pd
import json
import numpy as np
import geocoder
import math
from typing import Tuple
import time

In [68]:
with open('keys.json', 'r') as KeysFile:
    data = json.load(KeysFile)

FR24Key = data['FR24Key']
DEFAULT_LOCATION = [40.69131095346322, -74.38958047019509]
EARTH_RADIUS = 6371e3


In [69]:
def getLocation():
    lat, lon = geocoder.ip('me').latlng
    return lat, lon

In [70]:
def getBounds(radius_miles: float) -> Tuple[float, float, float, float]:
    """
    Return the (north_lat, south_lat, east_lon, west_lon) that bound
    a circle of *radius_m* centred on (lat_deg, lon_deg).

    All angles are in decimal degrees.  Longitudes are normalized to [-180, 180].
    """
    lat_deg, lon_deg = getLocation()
    EARTH_RADIUS_M = 6378137  # mean radius in metres (WGS-84)

    radius_meters = radius_miles * 1609.34 
    # Convert centre point to radians
    lat = math.radians(lat_deg)
    lon = math.radians(lon_deg)

    # Angular distance on the Earth’s surface
    ang_dist = radius_meters / EARTH_RADIUS_M      # radians

    # Latitude bounds are simple great-circle shifts north/south
    lat_north = lat + ang_dist
    lat_south = lat - ang_dist

    # Longitude bounds shrink with latitude (meridians converge toward the poles)
    # guard against cos(lat)=0 near the poles
    if abs(math.cos(lat)) < 1e-12:
        # At the poles every direction is “east/west”; set lon bounds to full range
        lon_east = math.pi
        lon_west = -math.pi
    else:
        delta_lon = math.asin(math.sin(ang_dist) / math.cos(lat))
        lon_east = lon + delta_lon
        lon_west = lon - delta_lon

    # Convert back to degrees and normalise longitudes to [-180, 180]
    def wrap(deg: float) -> float:
        """Wrap longitude from radians into the interval [-180, 180]°."""
        deg = math.degrees(deg)
        return (deg + 180) % 360 - 180

    north = str(round(math.degrees(lat_north), 3))
    south = str(round(math.degrees(lat_south), 3))
    east  = str(round(wrap(lon_east), 3))
    west  = str(round(wrap(lon_west), 3))

    return ','.join([north, south, west, east])


In [92]:
def getDistance(lat2, lon2):
    lat1, lon1 = getLocation()
    phi1 = lat1*np.pi/180           # φ, λ in radians
    phi2 = lat2*np.pi/180
    del_phi = (lat2 - lat1)*np.pi/180
    del_lambda = (lon2 - lon1)*np.pi/180

    a = np.sin(del_phi/2)*np.sin(del_phi/2) + \
        np.cos(phi1)*np.cos(phi2)*np.sin(del_lambda/2)*np.sin(del_lambda/2)

    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = EARTH_RADIUS*c;            # in meters
    return d/1600                  # convert to miles

In [ ]:
def getFlightsFR24(miles):
    url = "https://fr24api.flightradar24.com/api/live/flight-positions/full"
    params = {'bounds': getBounds(miles), 'altitude_ranges': '50-60000', 'categories': 'P,C,M,J,T'}
    headers = {'Accept': 'application/json',
    'Accept-Version': 'v1',
    'Authorization': f'Bearer {FR24Key}'
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json().get('data', [])
        df = pd.DataFrame(columns=['Airline', 'FlightNo', 'Type', 'Orig', 'Dest', 'Alt', 'Lat', 'Lon', 'Track', 'Timestamp', 'ETA'])
        for case in data:
            new_case = {'Airline': case.get('painted_as', ''),
                        'FlightNo': case.get('flight', ''),
                        'Type': case.get('type', ''),
                        'Orig': case.get('orig_iata', ''),
                        'Dest': case.get('dest_iata', ''),
                        'Alt': case.get('alt', ''),
                        'Lat': case.get('lat', ''),
                        'Lon': case.get('lon', ''),
                        'Track': case.get('track', ''),
                        'Timestamp': case.get('timestamp', ''),
                        'ETA': case.get('eta', '')
                        }
            df.loc[len(df)] = new_case
        
        df['Distance'] = round(getDistance(df['Lat'], df['Lon']), 1)
        return df.dropna(subset=['Airline', 'FlightNo', 'Orig'])
        
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

In [ ]:
df = getFlightsFR24(miles=20)
df = df.sort_values('Distance')

In [108]:
df

,Airline,FlightNo,Type,Orig,Dest,Alt,Lat,Lon,Track,Timestamp,ETA,Distance
19,UAL,UA1578,B772,EWR,LAX,4525,40.64470,-74.30194,283,2025-06-22T21:11:20Z,2025-06-23T01:31:44Z,6.4
5,EJA,EJA318,E55P,TEB,MKC,11825,40.78600,-74.46571,284,2025-06-22T21:11:19Z,2025-06-22T23:31:28Z,7.0
8,UAL,UA1808,A319,EWR,CLT,10150,40.61897,-74.49841,294,2025-06-22T21:11:19Z,2025-06-22T22:30:24Z,7.5
9,UAL,UA4193,CRJ7,EWR,AVL,5900,40.56413,-74.35468,201,2025-06-22T21:11:19Z,2025-06-22T22:32:32Z,9.7
13,UAL,UA1628,B39M,MBJ,EWR,5925,40.68828,-74.58856,32,2025-06-22T21:11:19Z,2025-06-22T21:16:49Z,9.9
15,UAL,UA1513,B737,EYW,EWR,4925,40.87262,-74.40033,53,2025-06-22T21:11:19Z,2025-06-22T21:15:48Z,12.1
2,UAL,UA2451,B38M,LGA,IAH,12275,40.86053,-74.20054,244,2025-06-22T21:11:19Z,2025-06-23T00:29:52Z,15.5
3,DAL,DL5837,E75S,BOS,CLT,32000,40.77585,-74.68525,230,2025-06-22T21:11:19Z,2025-06-22T22:26:08Z,15.9
18,UAL,UA212,B39M,PHX,EWR,4450,40.92572,-74.28778,79,2025-06-22T21:11:20Z,2025-06-22T21:15:49Z,16.9
1,JBU,B6524,A321,LAX,JFK,20350,40.93474,-74.31165,98,2025-06-22T21:11:19Z,2025-06-22T21:32:18Z,17.1


In [111]:
for row in df.itertuples():
    print(f"{row.Airline} {row.Type} from {row.Orig} to {row.Dest} at {row.Alt}ft {row.Distance} miles away")
    time.sleep(2)

UAL B772 from EWR to LAX at 4525ft 6.4 miles away
EJA E55P from TEB to MKC at 11825ft 7.0 miles away
UAL A319 from EWR to CLT at 10150ft 7.5 miles away
UAL CRJ7 from EWR to AVL at 5900ft 9.7 miles away
UAL B39M from MBJ to EWR at 5925ft 9.9 miles away
UAL B737 from EYW to EWR at 4925ft 12.1 miles away
UAL B38M from LGA to IAH at 12275ft 15.5 miles away
DAL E75S from BOS to CLT at 32000ft 15.9 miles away
UAL B39M from PHX to EWR at 4450ft 16.9 miles away
JBU A321 from LAX to JFK at 20350ft 17.1 miles away
UAL B738 from LGA to IAH at 16950ft 19.0 miles away
UAL B738 from ORD to LGA at 6425ft 20.2 miles away
UAL B38M from BDA to EWR at 3850ft 20.4 miles away
SWA B737 from MCI to LGA at 9975ft 20.9 miles away
UAL B738 from LGA to ORD at 9575ft 26.4 miles away
